In [1]:
from SagasuSubs.database import FileCrud,DialogCrud,dto,SeriesCrud,EpisodeCrud
from pathlib import Path

from functools import lru_cache

import requests

In [2]:
DB_PATH = Path('.') / 'db.sqlite3'

file_crud = FileCrud(DB_PATH)
dialog_crud =DialogCrud(DB_PATH)
series_crud = SeriesCrud(DB_PATH)
episode_crud = EpisodeCrud(DB_PATH)

In [3]:
@lru_cache(maxsize=16)
def serialize_serach_result(keyword:str):
    response = requests.get(f'http://127.0.0.1:3000/api/series/search?keyword={keyword}&fields=name&fields=name_cn&fields=description&from=0&size=20')
    data = response.json()
    for result in data['results']:
        return dto.SeriesCreate(id=result['info']['id'],name=result['info']['name'])
    return

serialize_serach_result('魔法少女小圆')

SeriesCreate(id=11017, name='魔法少女まどか☆マギカ')

In [4]:
@lru_cache(maxsize=16)
def get_series_episodes(id:int):
    response = requests.get(f'http://127.0.0.1:3000/api/series/{id}')
    data = response.json()
    return [dto.EpisodeCreate.parse_obj({**episode,'series_id':id}) for episode in data['episodes']]

get_series_episodes(11017)

[EpisodeCreate(id=131114, name='夢の中で会った、ような……', sort=1.0, type=0, series_id=11017),
 EpisodeCreate(id=131115, name='コネクト', sort=1.0, type=2, series_id=11017),
 EpisodeCreate(id=131116, name='Magia', sort=1.0, type=3, series_id=11017),
 EpisodeCreate(id=131117, name='それはとっても嬉しいなって', sort=2.0, type=0, series_id=11017),
 EpisodeCreate(id=131118, name='もう何も恐くない', sort=3.0, type=0, series_id=11017),
 EpisodeCreate(id=131119, name='奇跡も、魔法も、あるんだよ', sort=4.0, type=0, series_id=11017),
 EpisodeCreate(id=131120, name='後悔なんて あるわけない', sort=5.0, type=0, series_id=11017),
 EpisodeCreate(id=131121, name='こんなの絶対おかしいよ', sort=6.0, type=0, series_id=11017),
 EpisodeCreate(id=131122, name='本当の気持ちと向き合えますか？', sort=7.0, type=0, series_id=11017),
 EpisodeCreate(id=131123, name='あたしって、ほんとバカ', sort=8.0, type=0, series_id=11017),
 EpisodeCreate(id=131124, name='そんなの、あたしが許さない', sort=9.0, type=0, series_id=11017),
 EpisodeCreate(id=131125, name='もう誰にも頼らない', sort=10.0, type=0, series_id=11017),
 EpisodeCreate(id=13

In [5]:
try:
    series_crud.create_table()
    episode_crud.create_table()
except Exception as e:
    print(e)

(sqlite3.OperationalError) table series already exists
[SQL: 
CREATE TABLE series (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)

]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [10]:
from tqdm.notebook import tqdm
from typing import Dict

files = tqdm(file_crud)

for count,file in enumerate(files):
    file:dto.FileRead
    if not file.series_name:
        continue

    series = serialize_serach_result(file.series_name) 
    if not series:
        continue

    if not series_crud.read(series.id):
        series_crud.create(series)

    for episode in get_series_episodes(series.id):
        if not episode_crud.read(episode.id):
            episode_crud.create(episode)

    file.series_id = series.id
    file_crud.update(file)
    
    files.set_description(f'{file.id=}, {file.series_name}, {series.name}')


  0%|          | 0/41103 [00:00<?, ?it/s]